# Data Ingestion 

In [1]:
import os
import pandas as pd

In [2]:
os.getcwd()

'd:\\Model_Deployement\\End_to_End_Phishing_mail_detection_ML_project\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'd:\\Model_Deployement\\End_to_End_Phishing_mail_detection_ML_project'

This is the immutable class for storing only the path for dataingestion

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

 All the path varibales will be filled by configuration manager - which contains one function that fill out the data required for the path info.

Configuration manager will only fill out the immutavble class with their respective path variables and also create the necessary 

In [12]:
from Phising_mail_detection.utils.common import create_directories,read_yaml
from Phising_mail_detection.constants import *



# this configuration manager will have the constructor such that - 
# it will get the config file path, params and schema file paths for 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

print(CONFIG_FILE_PATH)


config\config.yaml


The main class which will do the stuff like  -  inheret the immutable class  this -> Data_ingestion_path_class and  use the path for downloading the data , storing it, preprocess it and unzip it etc .

In [13]:
from Phising_mail_detection import logger
from Phising_mail_detection.utils.common import get_size
import os
import gdown
import urllib.request as request
import zipfile
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

Now just like connect the classes with each other.

In [14]:
try:
    config= ConfigurationManager()
    config_path_data= config.get_data_ingestion_config()
    data_ingestion_main=DataIngestion(config=config_path_data)
    data_ingestion_main.download_file()
    data_ingestion_main.extract_zip_file()
except Exception as e:
    raise e

[2025-08-02 18:04:58,725:'INFO':common:yaml file: config\config.yaml loaded successfully]
[2025-08-02 18:04:58,730:'INFO':common:yaml file: params.yaml loaded successfully]
[2025-08-02 18:04:58,732:'INFO':common:yaml file: schema.yaml loaded successfully]
[2025-08-02 18:04:58,739:'INFO':common:created directory at: artifacts]
[2025-08-02 18:04:58,739:'INFO':common:created directory at: artifacts/data_ingestion]
[2025-08-02 18:04:58,747:'INFO':1000723594:Downloading data from https://drive.google.com/file/d/1RpSMHhd_AJHOHYZHeq_mskJwRLj4YbP3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1RpSMHhd_AJHOHYZHeq_mskJwRLj4YbP3
To: d:\Model_Deployement\End_to_End_Phishing_mail_detection_ML_project\artifacts\data_ingestion\data.zip
100%|██████████| 18.7M/18.7M [00:01<00:00, 13.0MB/s]

[2025-08-02 18:05:05,215:'INFO':1000723594:Downloaded data from https://drive.google.com/file/d/1RpSMHhd_AJHOHYZHeq_mskJwRLj4YbP3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
